# A2A Agent as Tool

A2A agents can be wrapped as tools within your agent's toolkit, similar to the Agents as Tools pattern but leveraging the A2A protocol for cross-platform communication.

In [ ]:
from uuid import uuid4
import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart
from strands import Agent, tool
from strands.models.openai import OpenAIModel

## A2A Agent Tool Class

Class-based approach to discover agent cards upfront and avoid repeated discovery calls.

In [ ]:
class A2AAgentTool:
    def __init__(self, agent_url: str, agent_name: str):
        self.agent_url = agent_url
        self.agent_name = agent_name
        self.agent_card = None
        self.client = None
        self.httpx_client = None

    async def initialize(self):
        self.httpx_client = httpx.AsyncClient(timeout=300)
        resolver = A2ACardResolver(httpx_client=self.httpx_client, base_url=self.agent_url)
        self.agent_card = await resolver.get_agent_card()

        config = ClientConfig(httpx_client=self.httpx_client, streaming=False)
        factory = ClientFactory(config)
        self.client = factory.create(self.agent_card)

    @tool
    async def call_agent(self, message: str) -> str:
        """
        Send a message to the A2A agent.

        Args:
            message: The message to send to the agent

        Returns:
            Response from the A2A agent
        """
        try:            
            msg = Message(
                kind="message",
                role=Role.user,
                parts=[Part(TextPart(kind="text", text=message))],
                message_id=uuid4().hex,
            )

            async for event in self.client.send_message(msg):
                if isinstance(event, tuple) and len(event) > 0:
                    task = event[0]
                    if hasattr(task, 'artifacts') and task.artifacts:
                        for artifact in task.artifacts:
                            if hasattr(artifact, 'parts') and artifact.parts:
                                response_text = ""
                                for part in artifact.parts:
                                    if hasattr(part, 'root') and hasattr(part.root, 'text'):
                                        response_text += part.root.text
                                if response_text:
                                    return response_text

            return f"No response received from {self.agent_name}"

        except Exception as e:
            return f"Error contacting {self.agent_name}: {str(e)}"

## Model Configuration

In [ ]:
model = OpenAIModel(
    client_args={
        "api_key": "none",
        "base_url": "http://localhost:1234/v1",
    },
    model_id="local-model",
    params={
        "temperature": 0.1,
        "top_p": 0.9,
        "frequency_penalty": 0.0
    }
)

## Initialize A2A Agent Tools

In [ ]:
calculator_agent = A2AAgentTool("http://localhost:9000", "Calculator Agent")

await calculator_agent.initialize()

## Create Orchestrator Agent

In [ ]:
orchestrator = Agent(
    model=model,
    tools=[calculator_agent.call_agent],
    system_prompt="You must always use the available A2A agent tools for any calculation, even simple ones. Never calculate directly."
)

## Test the Orchestrator

In [ ]:
orchestrator("Use the calculator agent to compute 2+2")